In [1]:
shp = '/mnt/c/mystuff/rms/results/rms_mun_v7.shp'

cols = {
    'dsidosos' : {
        "shpcols" : {'uid' : 'id'},
        "constants" : {"ano" : '2021', "id_indicator" : 'AG_IDOSOS'},
        "valcol" : "valor",
        "csv" : "ag_idosos_mun"
    },
    'dscreche' : {
        "shpcols" : {'uid' : 'id'},
        "constants" : {"ano" : '2021', "id_indicator" : 'AG_CRECHE'},
        "valcol" : "valor",
        "csv" : "ag_creche_mun"
    },
    'distevu' : {
        "shpcols" : {'uid' : 'id'},
        "constants" : {"ano" : '2021', "id_indicator" : 'AG_VERDE'},
        "valcol" : "valor",
        "csv" : "ag_verde_mun"
    },
    'disthsp' : {
        "shpcols" : {'uid' : 'id'},
        "constants" : {"ano" : '2021', "id_indicator" : 'AG_H'},
        "valcol" : "valor",
        "csv" : "ag_h_mun"
    },
    'pruido' : {
        "shpcols"   : {'uid' : 'id'},
        "constants" : {"ano" : '2021', "id_indicator" : 'P_RUIDO'},
        "valcol"    : "valor",
        "csv"       : "ruido65_mun",
        "filter"    : ['>', -1]
    }
}

ofolder = '/mnt/c/mystuff/rms/results/csv'

In [2]:
import os
import pandas as pd
from glass.rd.shp import shp_to_obj
from glass.wt import obj_to_tbl

In [3]:
# Open Shapefile

gdf = shp_to_obj(shp)

/home/jasp/.virtualenvs/glassenv/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [4]:
for k in cols:
    # Copy Dataframe
    csv_df = gdf.copy(deep=True)
    
    # Filter if necessary
    if "filter" in cols[k]:
        if cols[k]["filter"][0] == '>':
            csv_df = csv_df[csv_df[k] > cols[k]["filter"][1]]
    
    # Place constants
    for c in cols[k]['constants']:
        csv_df[c] = cols[k]['constants'][c] 
    
    # Delete columns
    mcols = [k] + list(
        cols[k]['constants'].keys()
    ) + list(cols[k]['shpcols'].keys())
    dcols = [c for c in csv_df.columns.values if c not in mcols]
    csv_df.drop(dcols, axis=1, inplace=True)
    
    # Rename columns
    cols[k]['shpcols'][k] = cols[k]['valcol']
    csv_df.rename(columns=cols[k]['shpcols'], inplace=True)
    
    # Round
    csv_df[cols[k]['valcol']] = csv_df[cols[k]['valcol']].round(decimals=5)
    
    # To CSV
    csv_df = pd.DataFrame(csv_df)
    obj_to_tbl(csv_df, os.path.join(ofolder, f"{cols[k]['csv']}.csv"), delimiter=';')